In [16]:
import wordsfromweb as ww
import pandas as pd
import re
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from time import sleep

In [2]:
pd.set_option('max_columns', 30)
pd.set_option('max_colwidth', 50)

In [3]:
ww.URLS['articles']

'http://bbc.com/mundo'

In [4]:
theurl = ww.URLS['articles']
s = ww.myget(theurl)
ll = ww.links_list(ww.article_links(s))
ldf = pd.DataFrame(ll, columns=['url','text'])
ldf['text'] = ldf['text'].str.strip()
ldf['furl'] = ldf['url'].apply(lambda x: x if x[0] =='h' else (theurl+x).replace('mundo/mundo','mundo')   )
link = ldf['furl'][0]

In [5]:
ldf.sample(10).sort_index()

,url,text,furl
13,/mundo/noticias-58079921,Por qué es tan difícil tener ideas originales ...,http://bbc.com/mundo/noticias-58079921
16,/mundo/noticias-58317688,5 mitos y creencias falsas sobre las alergias ...,http://bbc.com/mundo/noticias-58317688
18,/mundo/noticias-58364672,La isla más al norte del planeta que científic...,http://bbc.com/mundo/noticias-58364672
24,/mundo/noticias-58272444,"Video, Cómo una joven india superó sus miedos ...",http://bbc.com/mundo/noticias-58272444
25,/mundo/noticias-internacional-58260453,“No habrá venganza”: la llamada de un portavoz...,http://bbc.com/mundo/noticias-internacional-58...
28,/mundo/noticias-internacional-58351707,"""Haremos que paguen por esto"": Biden pide a su...",http://bbc.com/mundo/noticias-internacional-58...
33,/mundo/noticias-58329827,El joven que apareció de bebé en la portada de...,http://bbc.com/mundo/noticias-58329827
37,/mundo/noticias-internacional-58336737,La polémica en Argentina por la construcción d...,http://bbc.com/mundo/noticias-internacional-58...
42,/mundo/noticias-58363092,"Qué es el síndrome de Turner, la rara enfermed...",http://bbc.com/mundo/noticias-58363092
50,/mundo/noticias-internacional-58218335,Haití: imágenes de la destrucción causada por ...,http://bbc.com/mundo/noticias-internacional-58...


In [6]:
ldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     52 non-null     object
 1   text    52 non-null     object
 2   furl    52 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


### how can we save the articles so that we don't have to load them all the time while testing???

In [6]:
#moved into the module
import shelve# no longer needed
from bs4 import BeautifulSoup # no longer needed?

def save_articles(article_dict):
    s = shelve.open('artdict')
    for k in article_dict.keys():
        s[k] = article_dict[k].prettify()
    s.close()

def load_saved_articles():

    d = dict()
    s=shelve.open('artdict')
    for k in s.keys():
        d[k] = BeautifulSoup(s[k]).find() #get it back to the same type as the original
    #
    s.close()
    return d

In [15]:
%%time
def get_more_articles(ldf)
    article_dict = dict()
    for i, row in ldf.iterrows():#replace with an applied function at some point?
        #if i > 5: break #limit to a few urls for testing
        s2 = None
        s2 = ww.myget(row['furl'])
        e = s2.find('main')
        #get rid of div tag noise
        for x in e.find_all('div'):
            x.replace_with_children()
        article_dict[row['text'] ] = e #row['furl'] #ultimately this should hold the 'main' part of the article
        sleep(1)
        ww.save_articles(article_dict)
    return article_dict

article_dict = get_more_articles(ldf)
    
len(article_dict)

50

In [17]:
#regular articles typically have 2 sections
#links to parent topics typically have 7+ sections
for k in article_dict.keys():
    print(f"article title {k} - length is {len(article_dict[k])}; and has {len(article_dict[k].find_all('section'))} sections") 

article title Hay Festival - length is 10; and has 9 sections
article title Centroamérica Cuenta - length is 7; and has 6 sections
article title El "extremadamente peligroso" huracán Ida toca tierra en el sur de Estados Unidos - length is 39; and has 2 sections
article title "Tenía los ingredientes de una tormenta asesina": las condiciones que hicieron de Katrina el huracán más destructivo de la historia de EE.UU. - length is 76; and has 2 sections
article title "Mi disfraz en el aeropuerto de Kabul y cómo escapé de Afganistán" - length is 97; and has 2 sections
article title Qué significa que el Talibán defina a Afganistán como un Emirato Islámico - length is 60; and has 2 sections
article title La impactante imagen de más de 500 migrantes en una barca encontrados cerca de Italia - length is 19; and has 2 sections
article title Juanita Moody, la criptógrafa que fue clave para evitar una guerra nuclear entre EE.UU. y la Unión Soviética - length is 99; and has 2 sections
article title "

In [19]:
#if we want all the articles we've ever saved and not just the current set
d = ww.load_saved_articles()

D:\Users\Rob\Documents\python\git\rs_spanish\wordsfromweb.py:74: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 74 of the file D:\Users\Rob\Documents\python\git\rs_spanish\wordsfromweb.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  d[k] = BeautifulSoup(s[k]).find() #get it back to the same type as the original


In [20]:
[k for k in d.keys()]

['Hay Festival',
 'Centroamérica Cuenta',
 'Quiénes son las "Soñadoras afganas", el equipo adolescente de robótica que recibió asilo en México',
 'El "extremadamente peligroso" huracán Ida toca tierra en el sur de Estados Unidos',
 '"Tenía los ingredientes de una tormenta asesina": las condiciones que hicieron de Katrina el huracán más destructivo de la historia de EE.UU.',
 '"Mi disfraz en el aeropuerto de Kabul y cómo escapé de Afganistán"',
 'Qué significa que el Talibán defina a Afganistán como un Emirato Islámico',
 'La impactante imagen de más de 500 migrantes en una barca encontrados cerca de Italia',
 'Juanita Moody, la criptógrafa que fue clave para evitar una guerra nuclear entre EE.UU. y la Unión Soviética',
 '"Ojalá hubiera sabido antes que tenía el síndrome de congestión pélvica"',
 "La fascinante historia de Grace O'Malley, la intrépida y feroz reina pirata de Irlanda",
 'Qué son los "anillos de Einstein" que fotografió el Hubble y por qué son un fenómeno extraño en el es

In [51]:
#no longer used - we can just pull main tag from local shelf
def test_1_link(ldf,linkno=0):
    link = ldf.sample()['furl'].values[linkno]
    print(f"getting {link}")
    s2 = ww.myget(link)
    #article text is always in the main tag
    e = s2.find('main')
    print(e.prettify())
    return e

getting http://bbc.com/mundo/noticias-58323375


In [74]:
k=list(d.keys())[11]
e = d[k]
k

'Qué son los "anillos de Einstein" que fotografió el Hubble y por qué son un fenómeno extraño en el espacio'

In [71]:
if len(e.find_all('div')) > 0:
    #get rid of div tag noise
    for x in e.find_all('div'):
        x.replace_with_children()
else:
    print('div tags were already gone')

div tags were already gone


In [76]:
title = e.find('h1')
assert title.get_text().strip() == k

In [77]:
#locate all the sections in main
#in articles sections contain 'other articles to continue reading - maybe'
for sec in e.find_all('section'):
    header = sec.find(re.compile('^h[1-6]$'))
    if header is None:
        print('***** no header found in section')
        print(sec.get_text(strip=True))
    else:
        print(header.get_text(strip=True))

***** no header found in section

***** no header found in section
Saltar Quizás también te interese y continuar leyendoQuizás también te intereseCómo surgieron los agujeros negros más grandes del universoLa predicción de Stephen Hawking sobre los agujeros negros que científicos del MIT comprobaron 50 años despuésPor qué los errores son cruciales para el avance de la ciencia"Guerra de las galaxias hermanas": la espectacular imagen que captó el telescopio espacial HubbleFinal de Quizás también te interese


In [78]:
s = e.get_text(strip=True)
print(len(s), s[:300])

4213 Qué son los "anillos de Einstein" que fotografió el Hubble y por qué son un fenómeno extraño en el espacioRedacciónBBC News Mundo27 agosto 2021Fuente de la imagen,ESA/Hubble / NASAPie de foto,El anillo es una comprobación más de las teorías de la relatividad.Desde lo más remoto del universo llega un


In [55]:
#this wasn't the issue - commenting out
#is image alt text being turned into text? e.g. ...agosto 2021Fuente de la imagen, Getty ImagesEl bater...
#for x in e.find_all('img'):
#    x.decompose()
#remove image tags and then check the text again

In [96]:
#print(e.getText()) #no duplicatse but some text is concatenated without spacing

In [95]:
lines = []
for x in e.find_all('p'):
    lines.append(x.get_text()) #this method could preserve space but will get duplicates
len(lines)

23

In [80]:
article_dict = dict()

#### setting the text for an article

In [97]:
#using set to de-dupe loses ordering - using a Series instead
cur_art_txt = " ".join(pd.Series(lines).drop_duplicates())#de dupe and put all text 
article_dict[title.get_text()]=cur_art_txt

In [98]:
print(e.prettify())

<html>
 <body>
  <main role="main">
   <h1 class="bbc-1lsgtu3 e1yj3cbb0" id="content" tabindex="-1">
    Qué son los "anillos de Einstein" que fotografió el Hubble y por qué son un fenómeno extraño en el espacio
   </h1>
   <ul class="bbc-cdrraw e1c9i7u12" role="list">
    <li class="bbc-1a3w4ok e1c9i7u11" role="listitem">
     Redacción
    </li>
    <li class="bbc-1p92jtu e1c9i7u10" role="listitem">
     BBC News Mundo
    </li>
   </ul>
   <time class="bbc-14xtggo e4zesg50" datetime="2021-08-27">
    27 agosto 2021
   </time>
   <figure class="bbc-1qdcvv9 e6bmn90">
    <img alt="anillo" class="eehpdyc0 bbc-oabvne e1enwo3v0" height="549" src="https://ichef.bbci.co.uk/news/640/cpsprodpb/1710F/production/_120297449_210824-einstein-ring-al-1001-ab3248.jpg" srcset="https://ichef.bbci.co.uk/news/240/cpsprodpb/1710F/production/_120297449_210824-einstein-ring-al-1001-ab3248.jpg 240w, https://ichef.bbci.co.uk/news/320/cpsprodpb/1710F/production/_120297449_210824-einstein-ring-al-1001-ab3248.

general issues:
- quoted text in another language (e.g. English song titles)

## NLTK
issues
- proper nouns
- names with apostrophe e.g. O'Malley are split

### tokenize
try whitespace tokenizer

In [44]:
from nltk.tokenize import WhitespaceTokenizer
tokens = WhitespaceTokenizer().tokenize(cur_art_txt)
print(tokens[:200])#retains quotes and punctuation

['Fuente', 'de', 'la', 'imagen,', 'Getty', 'Images', 'A', 'diferencia', 'de', 'otros', 'casos', 'de', 'piratas,', 'no', 'es', 'fácil', 'encontrar', 'retratos', 'de', 'Grace', "O'Malley.", 'Pero', 'en', 'el', 'siglo', 'XX', 'artistas', 'como', 'el', 'estadounidense', 'Walter', 'Martin', 'Baumhofer', 'empiezan', 'a', 'imaginarla.', 'Grace', "O'Malley", 'había', 'dado', 'a', 'luz', 'unas', 'horas', 'antes,', 'pero', 'al', 'salir', 'a', 'la', 'cubierta,', 'la', 'intrépida', 'reina', 'pirata', 'supo', 'que', 'no', 'tenía', 'tiempo', 'para', 'descansar.', 'Una', 'banda', 'de', 'otros', 'piratas', 'había', 'atacado', 'y', 'abordado', 'su', 'barco,', 'y', 'ella', 'tuvo', 'que', 'liderar', 'a', 'su', 'tripulación', 'contra', 'ellos', 'y', 'proteger', 'a', 'su', 'hijo', 'recién', 'nacido.', "O'Malley", 'agarró', 'su', 'espada', 'y', 'reunió', 'a', 'sus', 'hombres', 'para', 'un', 'contraataque.', 'Cuando', 'terminó', 'la', 'pelea,', 'había', 'capturado', 'al', 'otro', 'barco', 'pirata.', 'Cuando'

### try RegexpTokenizer to return words only

In [43]:
from nltk.tokenize import RegexpTokenizer
tokens = RegexpTokenizer(r'\w+').tokenize(" ".join(pd.Series(lines).drop_duplicates()))
print(tokens[:200])

['Fuente', 'de', 'la', 'imagen', 'Getty', 'Images', 'A', 'diferencia', 'de', 'otros', 'casos', 'de', 'piratas', 'no', 'es', 'fácil', 'encontrar', 'retratos', 'de', 'Grace', 'O', 'Malley', 'Pero', 'en', 'el', 'siglo', 'XX', 'artistas', 'como', 'el', 'estadounidense', 'Walter', 'Martin', 'Baumhofer', 'empiezan', 'a', 'imaginarla', 'Grace', 'O', 'Malley', 'había', 'dado', 'a', 'luz', 'unas', 'horas', 'antes', 'pero', 'al', 'salir', 'a', 'la', 'cubierta', 'la', 'intrépida', 'reina', 'pirata', 'supo', 'que', 'no', 'tenía', 'tiempo', 'para', 'descansar', 'Una', 'banda', 'de', 'otros', 'piratas', 'había', 'atacado', 'y', 'abordado', 'su', 'barco', 'y', 'ella', 'tuvo', 'que', 'liderar', 'a', 'su', 'tripulación', 'contra', 'ellos', 'y', 'proteger', 'a', 'su', 'hijo', 'recién', 'nacido', 'O', 'Malley', 'agarró', 'su', 'espada', 'y', 'reunió', 'a', 'sus', 'hombres', 'para', 'un', 'contraataque', 'Cuando', 'terminó', 'la', 'pelea', 'había', 'capturado', 'al', 'otro', 'barco', 'pirata', 'Cuando', '

In [170]:
#remove stopwords
from nltk.corpus import stopwords
stopwrds = stopwords.words('spanish')
tokens2 = [x for x in tokens if x not in stopwrds]

In [171]:
s = SnowballStemmer('spanish', ignore_stopwords=True)

st = list(map(s.stem, tokens2)) #stems names too
st

['fuent',
 'imag',
 'getty',
 'imag',
 'el',
 'bater',
 'rolling',
 'ston',
 'charli',
 'watts',
 'mur',
 'mart',
 '80',
 'años',
 'confirm',
 'public',
 'es',
 'inmens',
 'tristez',
 'anunci',
 'muert',
 'amad',
 'charli',
 'watts',
 'lee',
 'comunic',
 'divulg',
 'medi',
 'represent',
 'fallec',
 'hoy',
 'pacif',
 'hospital',
 'londr',
 'rod',
 'famili',
 'añad',
 'el',
 'comunic',
 'dic',
 'watts',
 'quer',
 'espos',
 'padr',
 'abuel',
 'grand',
 'bater',
 'gener',
 'pid',
 'respet',
 'intim',
 'famili',
 'amig',
 'moment',
 'dificil',
 'la',
 'band',
 'previst',
 'reanud',
 'gir',
 'ee',
 'uu',
 'septiembr',
 'despues',
 'vers',
 'oblig',
 'posterg',
 'año',
 'pas',
 'pandemi',
 'coronavirus',
 'final',
 'quizas',
 'interes',
 'a',
 'principi',
 'agost',
 'grup',
 'comunic',
 'stev',
 'jord',
 'sustitut',
 'watts',
 'gir',
 'estadounidens',
 'fuent',
 'imag',
 'pa',
 'medi',
 'los',
 'rolling',
 'ston',
 '1969',
 'charli',
 'watts',
 'miembr',
 'ston',
 'ener',
 '1963',
 'uni',
 'm

In [172]:
pd.Series(st).value_counts()[:10]

watts      21
ston       13
charli     11
band        8
años        7
rolling     6
bater       5
dij         5
the         5
toc         4
dtype: int64

In [143]:
#tokens = word_tokenize(" ".join(pd.Series(lines).drop_duplicates()))#includes punctuation

['Fuente',
 'de',
 'la',
 'imagen',
 ',',
 'Getty',
 'Images',
 'El',
 'baterista',
 'de',
 'los',
 'Rolling',
 'Stones',
 ',',
 'Charlie',
 'Watts',
 ',',
 'murió',
 'este',
 'martes',
 'a',
 'los',
 '80',
 'años',
 ',',
 'confirmó',
 'su',
 'publicista',
 '.',
 '``',
 'Es',
 'con',
 'inmensa',
 'tristeza',
 'que',
 'anunciamos',
 'la',
 'muerte',
 'de',
 'nuestro',
 'amado',
 'Charlie',
 'Watts',
 "''",
 ',',
 'se',
 'lee',
 'en',
 'el',
 'comunicado',
 'divulgado',
 'a',
 'los',
 'medios',
 'por',
 'su',
 'representante',
 '.',
 '``',
 'Falleció',
 'hoy',
 'pacíficamente',
 'en',
 'un',
 'hospital',
 'de',
 'Londres',
 ',',
 'rodeado',
 'de',
 'su',
 'familia',
 "''",
 ',',
 'añade',
 '.',
 'El',
 'comunicado',
 'también',
 'dice',
 'que',
 'Watts',
 'fue',
 '``',
 'un',
 'querido',
 'esposo',
 ',',
 'padre',
 'y',
 'abuelo',
 "''",
 'y',
 '``',
 'uno',
 'de',
 'los',
 'más',
 'grandes',
 'bateristas',
 'de',
 'su',
 'generación',
 "''",
 'y',
 'pide',
 'que',
 'se',
 'respete',
 'l

## dealing with proper nouns

In [45]:
#proper nouns - only english and russian are supported by pos_tag
from nltk.tag import pos_tag
pos_tag(tokens, lang='esp')

NotImplementedError: Currently, NLTK pos_tag only supports English and Russian (i.e. lang='eng' or lang='rus')

## SpaCy

### stemming an lemmatization
https://medium.com/mlearning-ai/nlp-03-lemmatization-and-stemming-using-spacy-b2829becceca

spaCy does not contain any function for stemming. spaCy provides lemmatization via dictionary lookup and each language has its own dictionary. 


In [129]:
import es_core_news_sm
nlp = es_core_news_sm.load()
# tf/idf classification would inherently remove stopwords because they would be frequent in every document
sw = nlp.Defaults.stop_words
    # 'to', 'and' and 'or' are very common - needed for context but not for frequency
sw = sw.union({'a', 'y', 'o'})
print(f"stop words length {len(sw)} -- all stopwords{sw}")

def process_art_text(cur_art_txt, sw=sw, nlp=nlp):
    doc = nlp(cur_art_txt)
    if doc.sentiment !=0:
        print(f"*** sentiment is nonzero: {doc.sentiment}")
    posdf = pd.DataFrame([(w.text, w.pos_, w.lemma_, w.lemma_.lower() in sw) for w in doc], columns=['word','POS','lemma', 'stopword'])
    return posdf
posdf = process_art_text(cur_art_txt)
posdf

stop words length 554 -- all stopwords{'suya', 'teneis', 'aquéllos', 'pudo', 'tuya', 'lleva', 'pasado', 'bastante', 'bien', 'parece', 'hacemos', 'sabemos', 'último', 'pocos', 'cosas', 'habla', 'mismo', 'propia', 'final', 'siguiente', 'encima', 'estos', 'principalmente', 'podrias', 'delante', 'empleais', 'dan', 'total', 'suyo', 'tengo', 'siempre', 'esas', 'tuyo', 'tenido', 'si', 'alrededor', 'parte', 'podrian', 'cual', 'podrían', 'mí', 'éstos', 'cuantos', 'eras', 'nueva', 'fui', 'quiere', 'mejor', 'haceis', 'trabajar', 'primer', 'informo', 'lejos', 'mia', 'puedo', 'creo', 'muchos', 'ahí', 'haber', 'soy', 'varios', 'tendrán', 'podrá', 'aseguró', 'podria', 'cuatro', 'ningunas', 'sino', 'y', 'intenta', 'solos', 'breve', 'vamos', 'muchas', 'tiene', 'mucha', 'trabajo', 'buena', 'esta', 'van', 'conseguir', 'emplean', 'empleo', 'sabe', 'ninguno', 'antes', 'donde', 'dejó', 'podrán', 'asi', 'o', 'sea', 'nuevos', 'detras', 'deprisa', 'nuestras', 'yo', 'luego', 'usas', 'usan', 'debido', 'arriba', 

,word,POS,lemma,stopword
0,\n\n,SPACE,\n\n,False
1,Fuente,PROPN,Fuente,False
2,de,ADP,de,True
3,la,DET,el,True
4,imagen,NOUN,imagen,False
...,...,...,...,...
745,nuestro,DET,nuestro,True
746,mejor,ADJ,mejor,True
747,contenido,NOUN,contenido,False
748,.,PUNCT,.,False


In [85]:
#SpaCy token properties
w = doc[0]
print([x for x in dir(w) if x[0] != '_'])
#['ancestors', 'check_flag', 'children', 'cluster', 'conjuncts', 'dep', 'dep_', 'doc', 'ent_id', 'ent_id_', 'ent_iob',
#'ent_iob_', 'ent_kb_id', 'ent_kb_id_', 'ent_type', 'ent_type_', 'get_extension', 'has_dep', 'has_extension', 
#'has_head', 'has_morph', 'has_vector', 'head', 'i', 'idx', 'iob_strings', 'is_alpha', 'is_ancestor', 'is_ascii', 
#'is_bracket', 'is_currency', 'is_digit', 'is_left_punct', 'is_lower', 'is_oov', 'is_punct', 'is_quote', 
#'is_right_punct', 'is_sent_end', 'is_sent_start', 'is_space', 'is_stop', 'is_title', 'is_upper', 'lang', 'lang_', 
#'left_edge', 'lefts', 'lemma', 'lemma_', 'lex', 'lex_id', 'like_email', 'like_num', 'like_url', 'lower', 'lower_', 
#'morph', 'n_lefts', 'n_rights', 'nbor', 'norm', 'norm_', 'orth', 'orth_', 'pos', 'pos_', 'prefix', 'prefix_', 'prob',
#'rank', 'remove_extension', 'right_edge', 'rights', 'sent', 'sent_start', 'sentiment', 'set_extension', 'set_morph', 
#'shape', 'shape_', 'similarity', 'subtree', 'suffix', 'suffix_', 'tag', 'tag_', 'tensor', 'text', 'text_with_ws',
#'vector', 'vector_norm', 'vocab', 'whitespace_']

['ancestors', 'check_flag', 'children', 'cluster', 'conjuncts', 'dep', 'dep_', 'doc', 'ent_id', 'ent_id_', 'ent_iob', 'ent_iob_', 'ent_kb_id', 'ent_kb_id_', 'ent_type', 'ent_type_', 'get_extension', 'has_dep', 'has_extension', 'has_head', 'has_morph', 'has_vector', 'head', 'i', 'idx', 'iob_strings', 'is_alpha', 'is_ancestor', 'is_ascii', 'is_bracket', 'is_currency', 'is_digit', 'is_left_punct', 'is_lower', 'is_oov', 'is_punct', 'is_quote', 'is_right_punct', 'is_sent_end', 'is_sent_start', 'is_space', 'is_stop', 'is_title', 'is_upper', 'lang', 'lang_', 'left_edge', 'lefts', 'lemma', 'lemma_', 'lex', 'lex_id', 'like_email', 'like_num', 'like_url', 'lower', 'lower_', 'morph', 'n_lefts', 'n_rights', 'nbor', 'norm', 'norm_', 'orth', 'orth_', 'pos', 'pos_', 'prefix', 'prefix_', 'prob', 'rank', 'remove_extension', 'right_edge', 'rights', 'sent', 'sent_start', 'sentiment', 'set_extension', 'set_morph', 'shape', 'shape_', 'similarity', 'subtree', 'suffix', 'suffix_', 'tag', 'tag_', 'tensor', 't

In [106]:
posdf[posdf['POS'] == 'PROPN'].drop_duplicates()
#questionable = galaxias

,word,POS,lemma,stopword
1,Fuente,PROPN,Fuente,False
7,ESA,PROPN,ESA,False
9,Hubble,PROPN,Hubble,False
11,NASA,PROPN,NASA,False
62,Einstein,PROPN,Einstein,False
74,Tierra,PROPN,Tierra,False
108,Agencia,PROPN,Agencia,False
109,Espacial,PROPN,Espacial,False
110,Europea,PROPN,Europea,False
146,galaxias,PROPN,galaxias,False


In [107]:
posdf['stopword'].value_counts()

False    335
True     329
Name: stopword, dtype: int64

In [108]:
#store the key with the dataframe and then join all the dataframes for all articles
posdf2 = posdf[posdf['stopword']==False]
posdf2.sample(10)

,word,POS,lemma,stopword
629,curva,NOUN,curva,False
237,\n \n,SPACE,\n \n,False
10,/,PUNCT,/,False
156,anillo,NOUN,anillo,False
539,presencia,NOUN,presencia,False
103,fotografía,NOUN,fotografía,False
547,explicó,VERB,explicar,False
653,app,NOUN,app,False
587,materia,NOUN,materia,False
308,afectaría,VERB,afectar,False


In [109]:
poswords = set(posdf['POS']) - {'PUNCT', 'PROPN','SPACE'}

In [111]:
posdf2 = posdf2[posdf2['POS'].isin(poswords)]
posdf2.sample(10)

,word,POS,lemma,stopword
207,alta,ADJ,alto,False
95,gravedad,NOUN,gravedad,False
543,energía,NOUN,energía,False
637,extendiéndose,VERB,extender él,False
587,materia,NOUN,materia,False
103,fotografía,NOUN,fotografía,False
377,cercanas,ADJ,cercano,False
539,presencia,NOUN,presencia,False
381,eclipsado,ADJ,eclipsado,False
372,comprobó,VERB,comprobar,False


In [112]:
#most common stemmed words
posdf2['lemma'].value_counts()[:15]

luz               9
espacio-tiempo    5
gravedad          5
teoría            5
anillo            4
fenómeno          4
año               4
galaxia           4
relatividad       4
punto             3
materia           3
curvatura         3
físico            3
imagen            3
brillante         3
Name: lemma, dtype: int64

### use the working methodology for all article text

In [130]:
%%time
alldf = pd.DataFrame()
for k in d:
    print(k)
    lines = []
    e = d[k]
    for x in e.find_all('p'):
        lines.append(x.get_text()) #this method could preserve space but will get duplicates
    if len(lines)< 10:
        continue

    title = e.find('h1')
    if title.get_text().strip().lower() != k.lower():
        print(f"ERROR: title '{title.get_text().strip()}' \n doesn't match key '{k}'")
        continue
    cur_art_txt = " ".join(pd.Series(lines).drop_duplicates())#de dupe and put all text 
    posdf = process_art_text(cur_art_txt)
    posdf2 = posdf[posdf['stopword']==False]
    poswords = set(posdf['POS']) - {'PUNCT', 'PROPN','SPACE'}
    posdf2 = posdf2[posdf2['POS'].isin(poswords)]
    posdf2['title'] = k
    alldf = alldf.append(posdf2)

alldf.info()

Hay Festival
ERROR: title 'Hay Festival Digital Querétaro @BBC Mundo' 
 doesn't match key 'Hay Festival'
Centroamérica Cuenta
Quiénes son las "Soñadoras afganas", el equipo adolescente de robótica que recibió asilo en México
ERROR: title 'Afganistán: las "Soñadoras afganas", el equipo adolescente de robótica que recibió asilo en México' 
 doesn't match key 'Quiénes son las "Soñadoras afganas", el equipo adolescente de robótica que recibió asilo en México'
El "extremadamente peligroso" huracán Ida toca tierra en el sur de Estados Unidos
ERROR: title 'Huracán Ida: la "extremadamente peligrosa" tormenta toca tierra en el sur de Estados Unidos' 
 doesn't match key 'El "extremadamente peligroso" huracán Ida toca tierra en el sur de Estados Unidos'
"Tenía los ingredientes de una tormenta asesina": las condiciones que hicieron de Katrina el huracán más destructivo de la historia de EE.UU.
ERROR: title 'Huracán Katrina: las condiciones que lo convirtieron en el huracán más destructivo de la hi

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12397 entries, 6 to 747
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   word      12397 non-null  object
 1   POS       12397 non-null  object
 2   lemma     12397 non-null  object
 3   stopword  12397 non-null  bool  
 4   title     12397 non-null  object
dtypes: bool(1), object(4)
memory usage: 496.4+ KB
Wall time: 5.41 s


In [131]:
alldf['title'].value_counts()

"Estás todo lastimado": mi autorretrato de hombre que vive en un cuerpo ilegal                                                   2105
La fascinante historia de Grace O'Malley, la intrépida y feroz reina pirata de Irlanda                                            765
Qué hay realmente detrás de la falta de trabajadores en Estados Unidos                                                            724
Juanita Moody, la criptógrafa que fue clave para evitar una guerra nuclear entre EE.UU. y la Unión Soviética                      676
Por qué el canibalismo era considerado una buena medicina en el Renacimiento                                                      638
¿Cómo funciona realmente la internet en Cuba y hasta qué punto puede EE.UU. "restaurar" la conexión?                              634
La polémica en Argentina por la construcción de un nuevo puerto británico en las islas Malvinas/Falklands                         633
Centroamérica Cuenta                                          

In [135]:
alldf['lemma'].describe()

count      12397
unique      4568
top       imagen
freq          84
Name: lemma, dtype: object

In [136]:
alldf['lemma'].value_counts()[:50]

imagen          84
país            73
año             71
persona         54
mundo           47
recibir         44
querer          36
pasar           35
vida            34
hora            33
llegar          33
isla            32
humano          31
gobierno        30
explicar        30
encontrar       29
problema        29
historia        28
contenido       28
libro           27
talibán         26
médico          26
descarga        25
tipo            25
deber           25
cuerpo          25
internet        25
puerto          24
luz             24
interesar       24
activa él       23
alimento        23
notificación    23
perdertir       23
versión         23
app             23
mujer           23
idea            23
parecer         22
hombre          22
forma           21
siglo           21
dejar           21
mirar           21
contar          21
seguir          20
permitir        20
grupo           20
señalar         20
tratar          20
Name: lemma, dtype: int64

In [138]:
alldf['lemma'].value_counts()[:50].index

Index(['imagen', 'país', 'año', 'persona', 'mundo', 'recibir', 'querer',
       'pasar', 'vida', 'hora', 'llegar', 'isla', 'humano', 'gobierno',
       'explicar', 'encontrar', 'problema', 'historia', 'contenido', 'libro',
       'talibán', 'médico', 'descarga', 'tipo', 'deber', 'cuerpo', 'internet',
       'puerto', 'luz', 'interesar', 'activa él', 'alimento', 'notificación',
       'perdertir', 'versión', 'app', 'mujer', 'idea', 'parecer', 'hombre',
       'forma', 'siglo', 'dejar', 'mirar', 'contar', 'seguir', 'permitir',
       'grupo', 'señalar', 'tratar'],
      dtype='object')